# This NB demonstrates how we can use methods from visualizations

We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

In [1]:
# python imports
import pandas as pd
import altair as alt

# imports to load relationships
import requests
from pandas.io.json import json_normalize

# our imports
%load_ext autoreload
%autoreload 2
%cd ../intervals/
from main import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## `plot_ngrams_heatmap`

First, we do the normal steps to get a ngrams dataframe.

In [2]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/"
prefix = "CRIM_Model_00"
files = ["08"] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()

Successfully imported.


,[Superius],Altus,Tenor,Bassus
4.0,"4, 1, 2, 2, -3",NaN,NaN,NaN
20.0,NaN,"4, 1, 2, 2, -3",NaN,NaN
36.0,NaN,NaN,"4, 1, 2, 2, -3",NaN
52.0,NaN,NaN,NaN,"4, 1, 2, 2, -3"
62.0,"-2, -2, -2, 2, -2",NaN,NaN,NaN


Can read the documentation with `help(viz.plot_ngrams_heatmap)`

In [3]:
help(viz.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module visualizations:

plot_ngrams_heatmap(ngrams_df, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param ngrams_duration: if not None, rely on durations in the
    df to calculate the durations of the ngrams.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



In [4]:
viz.plot_ngrams_heatmap(mel_ngrams,selected_patterns=[], voices=[])

alt.VConcatChart(...)

### Selecting patterns

We could only include some patterns in the heatmaps

First, I collected the top patterns and turned these patterns into a list.

In [5]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"1, 2, -2, -2, -2",13
"2, -2, -2, -2, -2",11
"1, 1, 2, -2, -2",9
"-2, -2, -2, 2, 2",8
"2, -3, 2, -2, -2",8
"-3, 2, -2, -2, -2",8
"2, -3, 2, -3, 2",7
"-2, -2, -2, -2, 2",7
"2, -3, 2, 2, 2",6
"2, 2, 1, 2, -2",6


In [6]:
mel_ngrams_top_patterns_list 

['1, 2, -2, -2, -2',
 '2, -2, -2, -2, -2',
 '1, 1, 2, -2, -2',
 '-2, -2, -2, 2, 2',
 '2, -3, 2, -2, -2',
 '-3, 2, -2, -2, -2',
 '2, -3, 2, -3, 2',
 '-2, -2, -2, -2, 2',
 '2, -3, 2, 2, 2',
 '2, 2, 1, 2, -2']

We would pass the list of patterns into the `patterns` parametter.

In [7]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

**Cross-checking output from the heatmap with the ngrams dataframe**

We can see that '-2, -2, -2, -2, -2' is a really popular pattern. Let's view its start points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly

In [8]:
mel_ngrams[mel_ngrams ==  '-2, -2, -2, -2, -2' ].stack()

147.0  [Superius]    -2, -2, -2, -2, -2
161.0  [Superius]    -2, -2, -2, -2, -2
219.0  Tenor         -2, -2, -2, -2, -2
676.0  [Superius]    -2, -2, -2, -2, -2
dtype: object

In [9]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=['-2, -2, -2, -2, -2' ])

alt.VConcatChart(...)

## `generate_ngrams_and_duration` allow us to create heatmap with durations!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [10]:
ngrams, ngrams_duration = viz.generate_ngrams_and_duration(model, mel, n=5)

In [11]:
help(viz.generate_ngrams_and_duration)

Help on function generate_ngrams_and_duration in module visualizations:

generate_ngrams_and_duration(model, df, n=3, exclude=['Rest'], interval_settings=('d', True, True), offsets='first')
    This method accept a model and a dataframe with the melody or notes
    and rests and generate an ngram (in columnwise and unit=0 setting)
    and a corresponding duration ngram
    :param model: an Imported Piece object.
    :param df: dataframe containing consecutive notes.
    :param n: accept any positive integers and would output ngrams of the corresponding sizes
    can't handle the n=-1 option (refer to getNgrams documentation for more)
    :param exclude: (refer to getNgrams documentation)
    :param interval_settings: (refer to getNgrams documentation)
    :param offsets: (refer to getNgrams documentation)
    :return: ngram and corresponding duration dataframe!



In [12]:
viz.plot_ngrams_heatmap(ngrams, ngrams_duration, selected_patterns=mel_ngrams_top_patterns_list , voices=[])

alt.VConcatChart(...)

## `plot_close_match_heatmap`

In [13]:
help(viz.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module visualizations:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: t

In [14]:
viz.plot_close_match_heatmap(mel_ngrams, '-2, -2, -2, -2, -2', selected_patterns=[], voices=[])

alt.Chart(...)

**Additional Clarification**:
    
- As we change the cutoff with the slider, the ngrams with scores lower than the cutoff would be hidden away.
- Levenshtein distance takes into account how many changes we need to make to turn a pattern into another. For example, pattern '2, 2, 1, 2' and pattern '2, 2, 1, 3' would have the same levenshtein score 25 when compared to pattern '2, 2, 1, 1' because we need to make one change at the fourth character of the two strings to turn it into '2, 2, 1, 1'

In [15]:
# use the durations calculated from above
viz.plot_close_match_heatmap(ngrams,'-2, -2, -2, -2, -2', ngrams_duration, selected_patterns=[], voices=[])

alt.Chart(...)

## `plot_comparison_heatmap`

This method can be used to plot a relationship/observation json

### For relationships

In [16]:
# data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
# df_relationships = pd.json_normalize(data_relationships)
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == 'CRIM_Model_0008'].copy()

In [17]:
df_relationships_test.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
871,871,http://crimproject.org/data/relationships/882/,882,Mechanical transformation,Imitative duo,False,False,False,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0008,Ave Maria,"54-65/1-2,1-2,1-2,1-2,1-2,1-3,1-4,3-4,3-4,3-4,...",http://crimproject.org/data/observations/1764/,1764,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0005_5,Missa Ave Maria: Agnus Dei,"16-20,25-29/3,3-4,3-4,3-4,3-4,1-2,1-2,1-2,1-2,..."
872,872,http://crimproject.org/data/relationships/883/,883,Non-mechanical transformation,Periodic entry,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0008,Ave Maria,"1-10/1,1-4,1-4,1-4,2-4,2-4,3-4,3-4,4,4/@1-3,@1...",http://crimproject.org/data/observations/1766/,1766,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0005_5,Missa Ave Maria: Agnus Dei,"1-8/1-4,1-4,1-4,1-4,1-4,3-4,3-4,3-4/@1-3+@all+..."
971,971,http://crimproject.org/data/relationships/982/,982,New material,Homorhythm,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0008,Ave Maria,1/1/@1-3,http://crimproject.org/data/observations/1964/,1964,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0005_5,Missa Ave Maria: Agnus Dei,"34-39/1+4,1+4,1+4,1+4,1+4,1+4/@1-4+@1-4,@1-3+@..."
972,972,http://crimproject.org/data/relationships/983/,983,Non-mechanical transformation,Soggetto,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0008,Ave Maria,"44-46/1,1,1/@3,@1-3,@1",http://crimproject.org/data/observations/1966/,1966,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0005_5,Missa Ave Maria: Agnus Dei,"41-45/3-4,3-4,1-4,1-2,1-2/@4+@2-4,@1-3+@1-3,@4..."
973,973,http://crimproject.org/data/relationships/984/,984,Non-mechanical transformation,"Fuga, Homorhythm",False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0008,Ave Maria,"46-48/1+3-4,1+3-4,1+4/@3+@2-4+@4,@1-3+@1-4+@1-...",http://crimproject.org/data/observations/1968/,1968,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0005_5,Missa Ave Maria: Agnus Dei,"45-46/1+3-4,1+3-4/@3-4+@2-3+@3-4,@1-4+@1-3+@1-4"


In [18]:
help(viz.plot_comparison_heatmap)

Help on function plot_comparison_heatmap in module visualizations:

plot_comparison_heatmap(df, ema_col, main_category='musical_type', other_category='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart for relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param main_category: name of the main category for the first bar chart.
    The chart would be colored accordingly (default='musical_type').
    :param other_category: name of the other category for the zeroth bar chart.
    (default='observer.name')
    :param heat_map_width: the width of the final heatmap (default=800)
    :param heat_map_height: the height of the final heatmap (default =300)
    :return

In [19]:
viz.plot_comparison_heatmap(df_relationships_test, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

86:.


ValueError: could not convert string to float: 

**We can command+click on the bars to open a new tab containing the information about the observations on the website**

### For observations

In [20]:
# data_observations = requests.get('http://crimproject.org/data/observations/').json()
# df_observations = pd.json_normalize(data_observations)
# df_observations.to_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [21]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')
df_observations_test = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0008'].copy()
df_observations_test.loc[4797].to_dict()

{'Unnamed: 0': 4797,
 'url': 'http://crimproject.org/data/observations/4799/',
 'id': 4799,
 'ema': '-53/1,1,1,1,1/@1-3,@1-4,@1-4,@1-4,@1',
 'musical_type': 'Soggetto',
 'mt_cf': False,
 'mt_cf_voices': nan,
 'mt_cf_dur': False,
 'mt_cf_mel': False,
 'mt_sog': True,
 'mt_sog_voices': '[Superius]',
 'mt_sog_dur': True,
 'mt_sog_mel': True,
 'mt_sog_ostinato': False,
 'mt_sog_periodic': False,
 'mt_csog': False,
 'mt_csog_voices': nan,
 'mt_csog_dur': False,
 'mt_csog_mel': False,
 'mt_cd': False,
 'mt_cd_voices': nan,
 'mt_fg': False,
 'mt_fg_voices': nan,
 'mt_fg_int': nan,
 'mt_fg_tint': nan,
 'mt_fg_periodic': False,
 'mt_fg_strict': False,
 'mt_fg_flexed': False,
 'mt_fg_sequential': False,
 'mt_fg_inverted': False,
 'mt_fg_retrograde': False,
 'mt_pe': False,
 'mt_pe_voices': nan,
 'mt_pe_int': nan,
 'mt_pe_tint': nan,
 'mt_pe_strict': False,
 'mt_pe_flexed': False,
 'mt_pe_flt': False,
 'mt_pe_sequential': False,
 'mt_pe_added': False,
 'mt_pe_invertible': False,
 'mt_id': False,


In [22]:
viz.plot_comparison_heatmap(df_observations_test, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

86:.


ValueError: could not convert string to float: 

## Generate networks with `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.
- The `segments` column in the interactive df shows the range of the measures the observation was found, for example: `['5-9', ['14-18']` means that the observation have been found from measure 5 to measure 9, and again from measure 14 to measure 18.

### Melodic interval for pens

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_int` (`mt` = musical type, `pe` = periodic entries, `int`= intervals) and we would choose `interval_type` to be `melodic intervals`. The table's corresponding ema column is `ema`.

In [23]:
pen_networks, pen_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_pe_int', 'melodic', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

**See all the available starting intervals**:

- the 'all' key means that this networks contains all of the pens.
- If you select a network with specific starting intervals (e.g: 8+), it only contains pens that start with 8+.

In [24]:
pen_networks

{'all': <class 'pyvis.network.Network'> |N|=79 |E|=409,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '5+': <class 'pyvis.network.Network'> |N|=12 |E|=18,
 '8+': <class 'pyvis.network.Network'> |N|=10 |E|=14,
 '1+': <class 'pyvis.network.Network'> |N|=20 |E|=26,
 '2+': <class 'pyvis.network.Network'> |N|=2 |E|=3,
 '4-': <class 'pyvis.network.Network'> |N|=4 |E|=4,
 '9+': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 '7+': <class 'pyvis.network.Network'> |N|=6 |E|=5,
 '8-': <class 'pyvis.network.Network'> |N|=6 |E|=65,
 '5-': <class 'pyvis.network.Network'> |N|=7 |E|=8,
 '4+': <class 'pyvis.network.Network'> |N|=6 |E|=18,
 '6-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '9-': <class 'pyvis.network.Network'> |N|=2 |E|=1}

In [25]:
# show all the patterns mapped onto one network
pen_networks['all'].show('../CRIM_Intervals_Notebooks/all_pen.html')

In [26]:
# show only one family of patterns mapped onto one network
pen_networks['8+'].show('../CRIM_Intervals_Notebooks/8+_pen.html')

### melodic intervals for fuga

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= musical type, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [27]:
fuga_networks, fuga_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

In [28]:
# we are interested in the patterns that start with 4-
fuga_networks['4-'].show('../CRIM_Intervals_Notebooks/4-_fuga.html')

In [29]:
fuga_networks

{'all': <class 'pyvis.network.Network'> |N|=905 |E|=2,715,
 '4-': <class 'pyvis.network.Network'> |N|=68 |E|=137,
 '8-': <class 'pyvis.network.Network'> |N|=108 |E|=184,
 '5-': <class 'pyvis.network.Network'> |N|=123 |E|=293,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '4+': <class 'pyvis.network.Network'> |N|=117 |E|=206,
 '1+': <class 'pyvis.network.Network'> |N|=112 |E|=203,
 '8+': <class 'pyvis.network.Network'> |N|=92 |E|=259,
 '5+': <class 'pyvis.network.Network'> |N|=128 |E|=255,
 '12+': <class 'pyvis.network.Network'> |N|=19 |E|=25,
 '2+': <class 'pyvis.network.Network'> |N|=30 |E|=86,
 '12-': <class 'pyvis.network.Network'> |N|=25 |E|=27,
 '15-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '6+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '15+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '2-': <class 'pyvis.network.Network'> |N|=3 |E|=4,
 '9+': <class 'pyvis.network.Network'> |N|=7 |E|=7,
 '7-': <class 'pyvis.network.Network'> |N|=3 |E|=2,
 '9-': <class 'pyvis

## `plot_relationship_network`

This method accepts a dataframe of CRIM relationships and generate a network of observations.

In the network:

- the nodes are the observations (labeled with the piece and measures they are in)
- the edges are the relationships. 
    - points from a model_observation to a derivative observation.
    - are weighted differently based on the type of the relationships.
    - are labeled with the relationship type
    - are colored based on the users' choice.

### WARNING: Edge color isn't always a correct indicator of the relationship type.

The color of the edges is derived from the nodes; therefore, they would not be colored correctly when:

1. color='derivative'

model1 --(quotation)--> model2

model3 --(new material)--> model2

because of the first relationship, model2 and the two edges pointing to it would be colored with quotation's color.

2. color='model'

model1 --(quotation)--> model2

model1 --(new material)--> model3

because of the first relationship, model1 and the two out edges from model1 would be colored with quotation's color

In this case, we should pay attention to the edges' thickness (different weights mean different relationship types) or hover over the edges to see the relationship type labels.

### Seeing the big picture with the color modes being `model` or `derivative`

We have two options for the `color` parameter. 

- `derivative` (default): the edges and nodes of derivative observations will be colored according to the relationships. 
- `model` (default): the edges and nodes of model observations will be colored according to the relationships. 

**Here I use df_relationships.head(300) instead of df_relationships because plotting all of the relationships usually takes really long**.

#### color=`derivative`

In [30]:
nt = viz.plot_relationship_network(df_relationships.head(300), color='derivative')
nt.show('../CRIM_Intervals_Notebooks/small_relationships_network_derivative.html')

#### color=`model`

In [31]:
nt2 = viz.plot_relationship_network(df_relationships.head(300), color='model')
nt2.show('../CRIM_Intervals_Notebooks/small_relationships_network_model.html')

### Making uses of the other parameters.

The two plots are too cluttered so we can't see the relationship_type between specific measures inside pieces. 

#### Selecting specific models with `selected_model_ids` and specific derivatives with`selected_derivative_ids` 

In [32]:
nt3 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_derivatives.html')

In [33]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], color='derivative')
nt4.show('../CRIM_Intervals_Notebooks/selected_model.html')

#### Plotting pieces that are relevant to a list of interested pieces with `selected_members`
Having some of interest pieces, we want to plot all pieces that happen to connect to these pieces some how.

In [34]:
nt5 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt5.show('../CRIM_Intervals_Notebooks/selected_model_families.html')

5 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0003_5, CRIM_Mass_0018_1, CRIM_Mass_0018_2 no longer exist in df because of other filtering options
